# Extracting Features with Transformers

The datasets we have used so far have been described in terms of features. In the previous notebooks, we used a transaction-centric dataset. However, ultimately this was just a different format for representing feature-based data.

There are many other types of datasets, including text, images, sounds, movies, or even real objects. Most data mining algorithms, however, rely on having numerical or categorical features. This means we need a way to represent these types before we input them into the data mining algorithm.

The key concepts:
- Extracting features from datasets
- Creating new features
- Selecting good features
- Creating your own transformer for custom datasets